In [2]:
import pandas as pd

from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('bert-base-nli-mean-tokens')


In [3]:
true = pd.read_csv('C:/Users/katya/Fake_News/News_dataset/True.csv') #, sep=',')
fake = pd.read_csv('C:/Users/katya/Fake_News/News_dataset/Fake.csv') #, sep=',')

In [ ]:
fakedata = pd.read_csv(r'C:/Users/katya/Fake_News/fake_emb.csv')

In [4]:
fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [5]:
sentences = fake['text'].unique()
print(fake['subject'].unique())

sentence_embeddings = model.encode(sentences)
print(sentence_embeddings)

['News' 'politics' 'Government News' 'left-news' 'US_News' 'Middle-east']
[[-0.13895035  0.8994344   1.074438   ... -0.01868473  0.21273224
   0.19466305]
 [-0.17545132  0.68111575 -0.10805042 ...  0.05777051 -0.53642714
   0.60426563]
 [-0.40847576  0.62385744  0.2036819  ... -0.1412912  -0.41535875
   0.2841165 ]
 ...
 [-0.22883432  0.33919078  1.2166295  ...  0.19542217  1.0420856
  -0.24773398]
 [-0.059028    0.4361713  -0.44829327 ... -0.15544735  0.58680886
  -0.47699788]
 [-0.43946758 -0.00981666  0.24674253 ... -0.24000856  0.5328737
   0.16389881]]


In [6]:
sentence_df = pd.DataFrame()
sentence_arr = []
embedding_arr = []

i = 0

for sentence, embedding in zip(sentences, sentence_embeddings):
    #print("Sentence:", sentence)
    #print("Embedding:", embedding)
    #print("")    
    sentence_arr.append(sentences[i])
    embedding_arr.append(embedding)
    i = i + 1

sentence_df['text'] = sentence_arr
sentence_df['emb'] = embedding_arr
#sentence_df['sources'] = sources
print(sentence_df)

                                                    text  \
0      Donald Trump just couldn t wish all Americans ...   
1      House Intelligence Committee Chairman Devin Nu...   
2      On Friday, it was revealed that former Milwauk...   
3      On Christmas day, Donald Trump announced that ...   
4      Pope Francis used his annual Christmas Day mes...   
...                                                  ...   
17450  21st Century Wire says All the world s a stage...   
17451    Randy Johnson 21st Century WireThe majority ...   
17452  Tune in to the Alternate Current Radio Network...   
17453  21st Century Wire says A new front has just op...   
17454  21st Century Wire says It s not that far away....   

                                                     emb  
0      [-0.13895035, 0.8994344, 1.074438, 0.13726228,...  
1      [-0.17545132, 0.68111575, -0.10805042, 0.03226...  
2      [-0.40847576, 0.62385744, 0.2036819, 0.2079738...  
3      [-0.663932, 0.5116654, 1.0639156, -0

In [7]:
fakedata = sentence_df

In [8]:
csvpath = 'C:/Users/katya/Fake_News/fake_emb.csv'
fakedata.to_csv(csvpath)    

In [9]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 6):

    sentences = data['text']

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['emb']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
#sentence_df['distance_from_centroid'] = sentence_df.apply(distance_from_centroid, axis=1)

clustering_question(sentence_df)

(                                                    text  \
 0      Donald Trump just couldn t wish all Americans ...   
 1      House Intelligence Committee Chairman Devin Nu...   
 2      On Friday, it was revealed that former Milwauk...   
 3      On Christmas day, Donald Trump announced that ...   
 4      Pope Francis used his annual Christmas Day mes...   
 ...                                                  ...   
 17450  21st Century Wire says All the world s a stage...   
 17451    Randy Johnson 21st Century WireThe majority ...   
 17452  Tune in to the Alternate Current Radio Network...   
 17453  21st Century Wire says A new front has just op...   
 17454  21st Century Wire says It s not that far away....   
 
                                                      emb  cluster  \
 0      [-0.13895035, 0.8994344, 1.074438, 0.13726228,...        3   
 1      [-0.17545132, 0.68111575, -0.10805042, 0.03226...        4   
 2      [-0.40847576, 0.62385744, 0.2036819, 0.2079738..

In [17]:
fakedata_cluster = sentence_df

In [18]:
fakedata_cluster.to_csv('C:/Users/katya/Fake_News/fake_cluster.csv')    

In [19]:
filter = sentence_df.cluster.eq(1)
#print(sentence_df[filter]['text'])

array = sentence_df[filter]['text']
print(array.values)
print(len(array))

['On Christmas day, Donald Trump announced that he would  be back to work  the following day, but he is golfing for the fourth day in a row. The former reality show star blasted former President Barack Obama for playing golf and now Trump is on track to outpace the number of golf games his predecessor played.Updated my tracker of Trump s appearances at Trump properties.71 rounds of golf including today s. At this pace, he ll pass Obama s first-term total by July 24 next year. https://t.co/Fg7VacxRtJ pic.twitter.com/5gEMcjQTbH  Philip Bump (@pbump) December 29, 2017 That makes what a Washington Post reporter discovered on Trump s website really weird, but everything about this administration is bizarre AF. The coding contained a reference to Obama and golf:  Unlike Obama, we are working to fix the problem   and not on the golf course.  However, the coding wasn t done correctly.The website of Donald Trump, who has spent several days in a row at the golf course, is coded to serve up the f